In [10]:
import os
import json
import torch
import matplotlib.pyplot as plt
import numpy as np

# 读取ID.txt文件，获取所有json文件的ID
def read_id_file(path_to_id_txt):
    with open(path_to_id_txt, 'r') as file:
        file_ids = file.read().splitlines()
    return file_ids

# 解析每个JSON文件，提取房间和物体信息
def parse_json_files(file_ids, base_path):
    rooms_data = []
    objects_data = []

    for file_id in file_ids:
        file_path = os.path.join(base_path, f'{file_id}.json')
        
        if os.path.exists(file_path):
            with open(file_path, 'r') as f:
                data = json.load(f)
            
            # 提取房间信息
            for room in data.get('rooms', []):
                bbox = room['bbox']
                xlo, ylo, zlo, xhi, yhi, zhi = bbox
                rooms_data.append({
                    'region_index': room['region_index'],
                    'level_index': room['level_index'],
                    'label': room['label'],
                    'xlo': xlo,
                    'ylo': ylo,
                    'xhi': xhi,
                    'yhi': yhi,
                    'zlo': zlo,
                    'zhi': zhi
                })
                
                # 提取房间内的物体信息
                for obj in room['objects']:
                    objects_data.append({
                        'object_index': obj['object_index'],
                        'region_index': obj['region_index'],
                        'category_index': obj['category_index'],
                        'px': obj['position'][0],
                        'py': obj['position'][1],
                        'pz': obj['position'][2]
                    })

    return rooms_data, objects_data

# 绘制2D图形，房间用矩形表示，物体用圆表示
def plot_rooms_and_objects(rooms_data, objects_data):
    if not rooms_data:
        print("No room data available to plot.")
        return

    fig, ax = plt.subplots()

    # 绘制房间
    for room in rooms_data:
        rect = plt.Rectangle((room['xlo'], room['ylo']),
                             room['xhi'] - room['xlo'],
                             room['yhi'] - room['ylo'],
                             fill=None, edgecolor='blue', linewidth=1.5)
        ax.add_patch(rect)
        ax.text((room['xlo'] + room['xhi']) / 2, 
                (room['ylo'] + room['yhi']) / 2, 
                room['label'], 
                ha='center', va='center')

    # 绘制物体
    for obj in objects_data:
        ax.plot(obj['px'], obj['py'], 'ro')
        ax.text(obj['px'], obj['py'], obj['object_index'], color='red', fontsize=8)

    ax.set_aspect('equal')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Rooms and Objects')

    # 检查rooms_data是否为空，并设置合理的坐标范围
    if rooms_data:
        ax.set_xlim(np.min([room['xlo'] for room in rooms_data]), np.max([room['xhi'] for room in rooms_data]))
        ax.set_ylim(np.min([room['ylo'] for room in rooms_data]), np.max([room['yhi'] for room in rooms_data]))
    else:
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)

    plt.show()

# 归一化函数
def normalize_features(x):
    min_vals = x.min(axis=0, keepdims=True)
    max_vals = x.max(axis=0, keepdims=True)
    normalized_x = (x - min_vals) / (max_vals - min_vals + 1e-5)  # 避免除以零
    return normalized_x

# 生成GNN输入数据
def generate_gnn_data(rooms_data, objects_data):
    # 生成节点特征 (x) - 位置数据
    x = []
    y = []
    edge_index = []
    
    # Rooms as nodes
    for i, room in enumerate(rooms_data):
        x.append([room['xlo'], room['ylo'], room['zlo']])  # 使用3D信息
        y.append(room['label'])  # Label作为y
        for j, other_room in enumerate(rooms_data):
            if i != j and room['region_index'] == other_room['region_index']:
                edge_index.append([i, j])
    
    # Objects as nodes
    offset = len(rooms_data)
    for i, obj in enumerate(objects_data):
        x.append([obj['px'], obj['py'], obj['pz']])
        y.append(obj['category_index'])  # Category作为y
        for j, room in enumerate(rooms_data):
            if obj['region_index'] == room['region_index']:
                edge_index.append([offset + i, j])

    # 转换为张量
    x = torch.tensor(x, dtype=torch.float)
    y = torch.tensor(y, dtype=torch.long)
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    # 对节点特征进行归一化
    x = normalize_features(x)

    return x, edge_index, y

# 主函数
if __name__ == "__main__":
    path_to_id_txt = 'ID.txt'  # 替换为实际路径
    base_path = 'output'  # JSON文件的实际存储路径

    # 读取文件ID并解析JSON文件
    file_ids = read_id_file(path_to_id_txt)
    rooms_data, objects_data = parse_json_files(file_ids, base_path)

    # 打印数据检查
    print("Rooms data:", rooms_data)
    print("Objects data:", objects_data)

    # 绘制房间和物体
    plot_rooms_and_objects(rooms_data, objects_data)

    # 生成GNN数据
    x, edge_index, y = generate_gnn_data(rooms_data, objects_data)

    # 输出GNN数据以验证
    print("x (Node features):", x)
    print("edge_index (Graph edges):", edge_index)
    print("y (Labels):", y)


Rooms data: []
Objects data: []
No room data available to plot.


IndexError: min(): Expected reduction dim 0 to have non-zero size.